In [ ]:
##Import required libraries and packages

import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sb
from tqdm import tqdm

import torchvision
import torch.nn.functional as F
from torch.nn import init
import torchvision.transforms as Tranforms
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn as nn
import matplotlib.gridspec as gridspec
from IPython.display import clear_output
%matplotlib inline

torch.set_default_device("cuda")

In [ ]:
##Mount Google Drive to save tensors

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Check if cuda is available

if torch.cuda.is_available(): 
  torch.set_default_device("cuda")
  device = 'cuda'
else:
  torch.set_default_device("cpu")
  device = 'cpu'

print(device)

cuda


In [ ]:
##Import required Datasets: CIFAR, STL10, SVHN

import torchvision
import torchvision.datasets as data
import torchvision.transforms as Transforms

Tflatten = Transforms.Lambda(lambda x: torch.flatten(x))
Tcuda = Transforms.Lambda(lambda x: x.to("cuda"))
Tfloat = Transforms.Lambda(lambda x: x.to(torch.float))

T = Transforms.Compose([Transforms.ToTensor(), Tfloat, Tflatten, Tcuda])
cifar_train = data.CIFAR10("./", train = True, download = True, transform=T)
cifar_dataloader = torch.utils.data.DataLoader(cifar_train)
print("CIFAR:", next(iter(cifar_dataloader))[0].shape)
print("CIFAR:", cifar_train.data.shape)

T = Transforms.Compose([Transforms.ToTensor(), Tfloat, Transforms.Normalize(mean = [0,0,0], std = [5,5,5]), Tflatten, Tcuda])
cifar_test = data.CIFAR10("./", train = False, download = True, transform=T)
cifar_dataloader = torch.utils.data.DataLoader(cifar_test)
print("CIFAR:", next(iter(cifar_dataloader))[0].shape)
print("CIFAR:", cifar_test.data.shape)

T = Transforms.Compose([Transforms.ToTensor(), Transforms.Resize((32,32)), Tfloat, Transforms.Normalize(mean = [0,0,0], std = [5,5,5]), Tflatten, Tcuda])
stl10_train = data.STL10("./", split = 'train', download = True, transform=T)
stl10_dataloader = torch.utils.data.DataLoader(stl10_train)
print("STL10:", next(iter(stl10_dataloader))[0].shape)
print("STL10:", stl10_train.data.shape)

T = Transforms.Compose([Transforms.ToTensor(), Transforms.Resize((32,32)), Tfloat, Transforms.Normalize(mean = [0,0,0], std = [5,5,5]), Tflatten, Tcuda])
stl10_test = data.STL10("./", split = 'test', download = True, transform=T)
stl10_dataloader = torch.utils.data.DataLoader(stl10_test)
print("STL10:", next(iter(stl10_dataloader))[0].shape)
print("STL10:", stl10_test.data.shape)

T = Transforms.Compose([Transforms.ToTensor(), Tfloat, Transforms.Normalize(mean = [0,0,0], std = [5,5,5]), Tflatten, Tcuda])
svhn_train = data.SVHN("./", split = 'train', download = True, transform=T)
svhn_dataloader = torch.utils.data.DataLoader(svhn_train)
print("SVHN:", next(iter(svhn_dataloader))[0].shape)
print("SVHN:", svhn_train.data.shape)

In [ ]:
##Define all functions required for the theoretical results (refer to Theorems)

def eigen_squared(cr, z):
  num = z * cr ** 2 + cr ** 2 + z * cr - 2*cr + 1
  den = 2*z**2*cr * np.sqrt(4*z*cr**2 + (1-cr + cr*z)**2)
  return num/den + (1-1/cr)/(2*z**2)

def scale(cr,z):
  return 0.5 + (1+z*cr-np.sqrt(4*z*cr**2 + (1-cr + cr*z)**2))/(2*cr)

def scale_squared(cr,z):
  return -0.5 + (1+cr+z*cr)/(2*np.sqrt(4*z*cr**2 + (1-cr + cr*z)**2))

def scale_both_squared(cr,z):
  return scale(cr,z) - z*scale_squared(cr,z)

def var(c,r,d,N):
  cr = r/N
  if c < 1:
    return r * (scale_both_squared(cr, 1/c) + scale_squared(cr,1/c))/(d*(1-c))
  else:
    return r * c * scale(cr,1) / (d*(c-1))

def calc_gen_error_new(M,ntrn,c,ntst,r,S1,L):
  gen_error = 0
  if c<1:
    gen_error += (torch.diag(1/(1+S1**2*c)) @ L).square().sum()/L.shape[1]
  if c>1:
    gen_error += (torch.diag(1/(1+S1**2)) @ L).square().sum()/L.shape[1]

  return gen_error + var(c,r,M,ntrn)

# Cov might need to be diagonal....
def calc_gen_error_new_new(M,ntrn,c,ntst,r,S1,Cov):
  gen_error = 0
  if c<1:
    gen_error += eigen_squared(r/ntrn, 1/c) * (Cov).square().sum()/(c**2)
  if c>1:
    gen_error += eigen_squared(r/ntrn, 1) * (Cov).square().sum()

  return gen_error + var(c,r,M,ntrn)

def calc_gen_error(M,ntrn,c,ntst,r,S1,L):
  gen_error = 0
  if c<1:
    gen_error += (torch.diag(1/(1+S1**2*c)) @ L).square().sum()/L.shape[1]
  if c>1:
    gen_error += (torch.diag(1/(1+S1**2)) @ L).square().sum()/L.shape[1]
  
  gen_error += calc_wnorm(c,r,S1)/M

  return gen_error 

def calc_gen_error_regression(M,ntrn,c,ntst,r,S1,L,betahat):
  gen_error = 0
  if c<1:
    gen_error += (betahat.T @ torch.diag(1/(1+S1**2*c)) @ L).square().sum()/L.shape[1]
  if c>1:
    gen_error += (betahat.T @ torch.diag(1/(1+S1**2)) @ L).square().sum()/L.shape[1]
  
  gen_error += calc_wnorm_regression(c,r,S1,betahat)/M

  return gen_error 

def calc_lower_bound(M,ntrn,c,ntst,r,S1,L,alpha):
  wnorm_root = calc_wnorm(c,r,S1).sqrt()
  if c<1:
    bias = (torch.diag(1/(1+S1**2*c)) @ L).square().sum().sqrt()
  if c>1:
    bias = (torch.diag(1/(1+S1**2)) @ L).square().sum().sqrt()
  
  bias = (bias-alpha*(wnorm_root+1))**2/L.shape[0]

  return bias + wnorm_root.square()/M

def calc_upper_bound(M,ntrn,c,ntst,r,S1,L,alpha):
  wnorm_root = calc_wnorm(c,r,S1).sqrt()
  if c<1:
    bias = (torch.diag(1/(1+S1**2*c)) @ L).square().sum().sqrt()
  if c>1:
    bias = (torch.diag(1/(1+S1**2)) @  L).square().sum().sqrt()
  
  bias = (bias+alpha*(wnorm_root+1))**2/L.shape[0]

  return bias + wnorm_root.square()/M

def calc_W_minus_I_norm(c,r,S1,M):
  wnorm = 0
  if c<1:
    for i in range(1):
      wnorm = wnorm +  (((c**2*(S1[i]**2 + S1[i]**4))/((1+S1[i]**2*c)**2*(1-c))).sqrt()+1)**2
  if c>1:
    for i in range(1):
      wnorm = wnorm +  (((c*S1[i]**2)/((1+S1[i]**2)*(c-1))).sqrt()+1).square()
  return wnorm

def calc_wnorm(c,r,S1):
  wnorm = 0
  if c<1:
    for i in range(r):
      wnorm = wnorm +  ((S1[i]**2 + S1[i]**4))/((1/c+S1[i]**2)**2*(1-c))
  if c>1:
    for i in range(r):
      wnorm = wnorm +  (c*S1[i]**2)/((1+S1[i]**2)*(c-1))

  return wnorm 

def calc_wnorm_regression(c,r,S1,betahat):
  wnorm = 0
  if c<1:
    for i in range(r):
      wnorm = wnorm +  betahat[i,0]**2 * (c**2*(S1[i]**2 + S1[i]**4))/((1+S1[i]**2*c)**2*(1-c))
  if c>1:
    for i in range(r):
      wnorm = wnorm +  betahat[i,0]**2 * (c*S1[i]**2)/((1+S1[i]**2)*(c-1))

  return wnorm

In [ ]:
#Define a folder named Denoising and subfolders named dataRanks and figures to store your data and figures

path1_rank = F"/content/drive/MyDrive/Denoising/dataRanks/"
path2_rank = F"/content/drive/MyDrive/Denoising/dataRanks/"

path_figures = F"/content/drive/My Drive/Denoising/figures/"

## Data augmentation without independence for multiple r values (Figure 2d)

In [ ]:
from numpy.lib.arraysetops import setxor1d

#Here dimension of dataset 'd' is the variable 'M'
M = 3072

N = torch.arange(1000,6000,50).to(torch.int) ##more c values for smoother theory curves

r_values = [25,50,100,150]
theta_values = torch.tensor([1])

Ntst = 2500

Err_stl10 = torch.zeros(len(r_values),theta_values.shape[0],N.shape[0]).to(device) #theoretical error
Err_emp_stl10 = torch.zeros(len(r_values),theta_values.shape[0],N.shape[0]//20).to(device) #emperical error 

Err_svhn = torch.zeros(len(r_values),theta_values.shape[0],N.shape[0]).to(device) #theoretical error
Err_emp_svhn = torch.zeros(len(r_values),theta_values.shape[0],N.shape[0]//20).to(device) #emperical error 

Err_cifar = torch.zeros(len(r_values),theta_values.shape[0],N.shape[0]).to(device) #theoretical error
Err_emp_cifar = torch.zeros(len(r_values),theta_values.shape[0],N.shape[0]//20).to(device) #emperical error 

#Number of runs
T = 10

for i,r in list(enumerate(r_values)):
  for j,_ in enumerate(theta_values):
    theta = 1
    #print(r,theta)
    for k in range(N.shape[0]):
      c = M/N[k]
      cifar_data = torch.utils.data.DataLoader(cifar_train, batch_size = N[0].item(), shuffle = False)
      batch = next(iter(cifar_data))[0].T
      Xtrn = torch.clone(batch)
      # for _ in range(k):
      #   Xtrn = torch.cat((Xtrn,batch),dim=1)

      cifar_test_data = torch.utils.data.DataLoader(cifar_test, batch_size = Ntst, shuffle = False)
      Xtst_cifar = next(iter(cifar_test_data))[0].T

      stl10_data = torch.utils.data.DataLoader(stl10_train, batch_size = Ntst, shuffle = False)
      Xtst_stl10 = next(iter(stl10_data))[0].T

      svhn_data = torch.utils.data.DataLoader(svhn_train, batch_size = Ntst, shuffle = False)
      Xtst_svhn = next(iter(svhn_data))[0].T

      # print(Xtrn.shape, Xtst_cifar.shape)

      #print(c)
      U,S,Vh = torch.linalg.svd(Xtrn)
      Xtrn = theta * U[:,:r] @ torch.diag(S[:r]) @ Vh[:r,:]

      P = U[:,:r] @ U[:,:r].T

      Xtst_cifar_proj = P @ Xtst_cifar
      Xtst_stl10_proj = P @ Xtst_stl10
      Xtst_svhn_proj = P @ Xtst_svhn

      L_cifar = U[:,:r].T @ Xtst_cifar
      L_stl10 = U[:,:r].T @ Xtst_stl10
      L_svhn = U[:,:r].T @ Xtst_svhn

      Err_cifar[i,j,k] = calc_gen_error(M,N[k],c,Ntst,r,theta*(N[k].item()/N[0].item())**0.5*S[:r],L_cifar)
      Err_stl10[i,j,k] = calc_gen_error(M,N[k],c,Ntst,r,theta*(N[k].item()/N[0].item())**0.5*S[:r],L_stl10)
      Err_svhn[i,j,k] = calc_gen_error(M,N[k],c,Ntst,r,theta*(N[k].item()/N[0].item())**0.5*S[:r],L_svhn)
      #print(Err_cifar[i,j,k])

      if k%20 == 0:  ##We compute empirical error only for N = {1000,2000,3000,4000,5000}
        Xtrn = torch.clone(batch)
        for _ in range(k//20):
          Xtrn = torch.cat((Xtrn,batch),dim=1)
        U,S,Vh = torch.linalg.svd(Xtrn)
        Xtrn = theta * U[:,:r] @ torch.diag(S[:r]) @ Vh[:r,:]

        for _ in tqdm(range(T)):
          Atrn = torch.randn_like(Xtrn)/np.sqrt(M)
          W = Xtrn.mm(torch.pinverse(Xtrn+Atrn))

          Atst_cifar = torch.randn_like(Xtst_cifar_proj)/np.sqrt(M)
          Yp = W.mm(Xtst_cifar_proj + Atst_cifar)
          Err_emp_cifar[i,j,k//20] += (Xtst_cifar_proj - Yp).square().sum()/(T*Ntst)

          Atst_stl10 = torch.randn_like(Xtst_stl10_proj)/np.sqrt(M)
          Yp = W.mm(Xtst_stl10_proj + Atst_stl10)
          Err_emp_stl10[i,j,k//20] += (Xtst_stl10_proj - Yp).square().sum()/(T*Ntst)

          Atst_svhn = torch.randn_like(Xtst_svhn_proj)/np.sqrt(M)
          Yp = W.mm(Xtst_svhn_proj + Atst_svhn)
          Err_emp_svhn[i,j,k//20] += (Xtst_svhn_proj - Yp).square().sum()/(T*Ntst)

      torch.save(Err_emp_cifar,path2_rank+"-data-aug-cifar-emp-ranks.pt")
      torch.save(Err_cifar,path1_rank+"-data-aug-cifar-ranks.pt")  
      torch.save(Err_emp_stl10,path2_rank+"-data-aug-stl10-emp-ranks.pt")
      torch.save(Err_stl10,path1_rank+"-data-aug-stl10-ranks.pt")  
      torch.save(Err_emp_svhn,path2_rank+"-data-aug-svhn-emp-ranks.pt")
      torch.save(Err_svhn,path1_rank+"-data-aug-svhn-ranks.pt")    

Generate figures for non-independed data augmentation

In [ ]:
#Define style parameters for figures

plt.rc('font',size=20)
plt.rc('xtick', labelsize=14) 
plt.rc('ytick', labelsize=14)
plt.rc('legend',fontsize=13)

In [ ]:
## create figure for data aug CIFAR dataset
device = 'cpu'

cmap = plt.get_cmap("viridis")
sliced_cmap = cmap(np.linspace(0.5,0.9,4))

Err_emp_ranks = torch.load(path2_rank+"-data-aug-cifar-emp-ranks.pt",map_location = device)
Err_ranks = torch.load(path1_rank+"-data-aug-cifar-ranks.pt",map_location = device)

M = 3072
N = torch.arange(1000,6000,1000).to(torch.int).cpu()
N_theory = torch.arange(1000,6000,50).to(torch.int).cpu()

Cinverse = (N/M).cpu().numpy()
Cinverse_theory = (N_theory/M).cpu().numpy()

for i in range(4):
  plt.plot(Cinverse_theory,Err_ranks[i,0,:],color=sliced_cmap[i])
  plt.plot(Cinverse,Err_emp_ranks[i,0,:],'.',markersize = 15,color=sliced_cmap[i],label='_nolegend_')


plt.yscale("log")
plt.xlabel("1/c = N/d")
plt.ylabel("Generalization Error")
plt.legend(['r = 25','r = 50','r = 100','r = 150'])
plt.savefig(path_figures+"CIFAR_error_dataAug_ranks.pdf", bbox_inches='tight', facecolor='white', dpi = 300, format = 'pdf')

In [ ]:
## create figure for data aug error stl10 dataset
device = 'cpu'

cmap = plt.get_cmap("viridis")
sliced_cmap = cmap(np.linspace(0,0.4,4))

Err_emp_ranks = torch.load(path2_rank+"-data-aug-stl10-emp-ranks.pt",map_location = device)
Err_ranks = torch.load(path1_rank+"-data-aug-stl10-ranks.pt", map_location = device)

M = 3072
N = torch.arange(1000,6000,1000).to(torch.int).cpu()
N_theory = torch.arange(1000,6000,50).to(torch.int).cpu()

Cinverse = (N/M).cpu().numpy()
Cinverse_theory = (N_theory/M).cpu().numpy()

for i in range(4):
  plt.plot(Cinverse_theory,Err_ranks[i,0,:],color=sliced_cmap[i])
  plt.plot(Cinverse,Err_emp_ranks[i,0,:],'.',markersize = 15,color=sliced_cmap[i],label='_nolegend_')


plt.yscale("log")
plt.xlabel("1/c = N/d")
plt.ylabel("Generalization Error")
plt.legend(['r = 25','r = 50','r = 100','r = 150'])
plt.savefig(path_figures+"stl10_error_dataAug_ranks.pdf", bbox_inches='tight', facecolor='white', dpi = 300, format = 'pdf')

In [ ]:
## create figure for data aug error svhn dataset
device = 'cpu'

cmap = plt.get_cmap("inferno")
sliced_cmap = cmap(np.linspace(0.5,0.9,4))

Err_emp_ranks = torch.load(path2_rank+"-data-aug-svhn-emp-ranks.pt",map_location = device)
Err_ranks = torch.load(path1_rank+"-data-aug-svhn-ranks.pt", map_location = device)

M = 3072
N = torch.arange(1000,6000,1000).to(torch.int).cpu()
N_theory = torch.arange(1000,6000,50).to(torch.int).cpu()

Cinverse = (N/M).cpu().numpy()
Cinverse_theory = (N_theory/M).cpu().numpy()

for i in range(4):
  plt.plot(Cinverse_theory,Err_ranks[i,0,:],color=sliced_cmap[i])
  plt.plot(Cinverse,Err_emp_ranks[i,0,:],'.',markersize = 15,color=sliced_cmap[i],label='_nolegend_')

plt.yscale("log")
plt.xlabel("1/c = N/d")
plt.ylabel("Generalization Error")
plt.legend(['r = 25','r = 50','r = 100','r = 150'])
plt.savefig(path_figures+"svhn_error_dataAug_ranks.pdf", bbox_inches='tight', facecolor='white', dpi = 300, format = 'pdf')

## Data augmentation without identicality for multiple r values (Figure 2h)

In [ ]:
from numpy.lib.arraysetops import setxor1d
## Non-identical data augmentation experiments

M = 3072
N = torch.arange(1050,10500,550).to(torch.int)

r_values = [25,50,100,150]

Ntst = 2500

Err_stl10 = torch.zeros(len(r_values),N.shape[0]).to(device) #theoretical error
Err_emp_stl10 = torch.zeros(len(r_values),N.shape[0]).to(device) #emperical error 

Err_svhn = torch.zeros(len(r_values),N.shape[0]).to(device) #theoretical error
Err_emp_svhn = torch.zeros(len(r_values),N.shape[0]).to(device) #emperical error 

Err_cifar = torch.zeros(len(r_values),N.shape[0]).to(device) #theoretical error
Err_emp_cifar = torch.zeros(len(r_values),N.shape[0]).to(device) #emperical error 

T = 200 #Number of runs

for i,r in list(enumerate(r_values)):
  print(r)
  for j in range(N.shape[0]):
    c = M/N[j]
    cifar_train_data = torch.utils.data.DataLoader(cifar_train, batch_size = (N[j]//2).item(), shuffle = False)
    Xtrn_cifar = next(iter(cifar_train_data))[0].T

    stl10_data = torch.utils.data.DataLoader(stl10_train, batch_size = (N[j]//2).item(), shuffle = False)
    Xtrn_stl10 = next(iter(stl10_data))[0].T

    Xtrn = torch.cat((Xtrn_cifar,Xtrn_stl10),1) ##concatenating the training data for order d x N

    cifar_test_data = torch.utils.data.DataLoader(cifar_test, batch_size = Ntst, shuffle = False)
    Xtst_cifar = next(iter(cifar_test_data))[0].T

    stl10_test_data = torch.utils.data.DataLoader(stl10_test, batch_size = Ntst, shuffle = False)
    Xtst_stl10 = next(iter(stl10_test_data))[0].T

    svhn_data = torch.utils.data.DataLoader(svhn_train, batch_size = Ntst, shuffle = False)
    Xtst_svhn = next(iter(svhn_data))[0].T

    # print(Xtrn.shape, Xtrn_cifar.shape, Xtst_stl10.shape)

    # print(c)

    U,S,Vh = torch.linalg.svd(Xtrn)
    Xtrn = U[:,:r] @ torch.diag(S[:r]) @ Vh[:r,:]

    P = U[:,:r] @ U[:,:r].T

    Xtst_cifar_proj = P @ Xtst_cifar
    Xtst_stl10_proj = P @ Xtst_stl10
    Xtst_svhn_proj = P @ Xtst_svhn

    L_cifar = U[:,:r].T @ Xtst_cifar
    L_stl10 = U[:,:r].T @ Xtst_stl10
    L_svhn = U[:,:r].T @ Xtst_svhn

    Err_cifar[i,j] = calc_gen_error(M,N[j],c,Ntst,r,S[:r],L_cifar)
    Err_stl10[i,j] = calc_gen_error(M,N[j],c,Ntst,r,S[:r],L_stl10)
    Err_svhn[i,j] = calc_gen_error(M,N[j],c,Ntst,r,S[:r],L_svhn)
    
    for k in tqdm(range(T)):
        Atrn = torch.randn_like(Xtrn)/np.sqrt(M)
        W = Xtrn.mm(torch.pinverse(Xtrn+Atrn))

        Atst_cifar = torch.randn_like(Xtst_cifar_proj)/np.sqrt(M)
        Yp = W.mm(Xtst_cifar_proj + Atst_cifar)
        Err_emp_cifar[i,j] += (Xtst_cifar_proj - Yp).square().sum()/(T*Ntst)

        Atst_stl10 = torch.randn_like(Xtst_stl10_proj)/np.sqrt(M)
        Yp = W.mm(Xtst_stl10_proj + Atst_stl10)
        Err_emp_stl10[i,j] += (Xtst_stl10_proj - Yp).square().sum()/(T*Ntst)

        Atst_svhn = torch.randn_like(Xtst_svhn_proj)/np.sqrt(M)
        Yp = W.mm(Xtst_svhn_proj + Atst_svhn)
        Err_emp_svhn[i,j] += (Xtst_svhn_proj - Yp).square().sum()/(T*Ntst)

    # print((Err_emp_cifar[i,j]-Err_cifar[i,j]).abs()/Err_emp_cifar[i,j])
    # print((Err_emp_stl10[i,j]-Err_stl10[i,j]).abs()/Err_emp_stl10[i,j])
    # print((Err_emp_svhn[i,j]-Err_svhn[i,j]).abs()/Err_emp_svhn[i,j])

    torch.save(Err_emp_cifar,path2_rank+"cifar-emp-nonIdentical-ranks.pt")
    torch.save(Err_cifar,path1_rank+"cifar-nonIdentical-ranks.pt")  
    torch.save(Err_emp_stl10,path2_rank+"stl10-emp-nonIdentical-ranks.pt")
    torch.save(Err_stl10,path1_rank+"stl10-nonIdentical-ranks.pt")  
    torch.save(Err_emp_svhn,path2_rank+"svhn-emp-nonIdentical-ranks.pt")
    torch.save(Err_svhn,path1_rank+"svhn-nonIdentical-ranks.pt")    

Generate figures for non-identical Data Augmentation experiments

In [ ]:
plt.rc('font',size=20)
plt.rc('xtick', labelsize=14) 
plt.rc('ytick', labelsize=14)
plt.rc('legend',fontsize=15)

In [ ]:
## create figure for non-Identical training data - CIFAR dataset
device = 'cpu'

cmap = plt.get_cmap("viridis")
sliced_cmap = cmap(np.linspace(0.5,0.9,4))

Err_emp_cifar_ranks = torch.load(path2_rank+"cifar-emp-nonIdentical-ranks.pt",map_location = device)
Err_cifar_ranks = torch.load(path1_rank+"cifar-nonIdentical-ranks.pt",map_location = device)

M = 3072
N = torch.arange(1050,10500,550).to(torch.int).cpu()

Cinverse = (N/M).cpu().numpy()

for i in range(4):
  plt.plot(Cinverse,Err_cifar_ranks[i,:],color=sliced_cmap[i])
  plt.plot(Cinverse,Err_emp_cifar_ranks[i,:],'.',markersize = 15,color=sliced_cmap[i],label='_nolegend_')

plt.yscale("log")
plt.xlabel("1/c = N/d")
plt.ylabel("Generalization Error")
plt.legend(['r = 25','r = 50','r = 100','r = 150'])
plt.savefig(path_figures+"CIFAR_error_nonIdentical_ranks.pdf", bbox_inches='tight', facecolor='white', dpi = 300, format = 'pdf')

In [ ]:
## create figure for non-Identical training set - error stl10 dataset

cmap = plt.get_cmap("viridis")
sliced_cmap = cmap(np.linspace(0,0.4,4))

device = 'cpu'

Err_emp_stl10_ranks = torch.load(path2_rank+"stl10-emp-nonIdentical-ranks.pt",map_location = device)
Err_stl10_ranks = torch.load(path1_rank+"stl10-nonIdentical-ranks.pt",map_location = device)

M = 3072
N = torch.arange(1050,10500,550).to(torch.int).cpu()

Cinverse = (N/M).cpu().numpy()

for i in range(4):
  plt.plot(Cinverse,Err_stl10_ranks[i,:],color=sliced_cmap[i])
  plt.plot(Cinverse,Err_emp_stl10_ranks[i,:],'.',markersize = 15,color=sliced_cmap[i],label='_nolegend_')

plt.yscale("log")
plt.xlabel("1/c = N/d")
plt.ylabel("Generalization Error")
plt.legend(['r = 25','r = 50','r = 100','r = 150'])
plt.savefig(path_figures+"STL10_error_nonIdentical_ranks.pdf", bbox_inches='tight', facecolor='white', dpi = 300, format = 'pdf')

In [ ]:
## create figure for non-Identical training data - SVHN dataset

cmap = plt.get_cmap("inferno")
sliced_cmap = cmap(np.linspace(0.5,0.9,4))

device = 'cpu'

Err_emp_svhn_ranks = torch.load(path2_rank+"svhn-emp-nonIdentical-ranks.pt",map_location = device)
Err_svhn_ranks = torch.load(path1_rank+"svhn-nonIdentical-ranks.pt",map_location = device)

M = 3072
N = torch.arange(1050,10500,550).to(torch.int).cpu()

Cinverse = (N/M).cpu().numpy()

for i in range(4):
  plt.plot(Cinverse,Err_svhn_ranks[i,:],color=sliced_cmap[i])
  plt.plot(Cinverse,Err_emp_svhn_ranks[i,:],'.',markersize = 15,color=sliced_cmap[i],label='_nolegend_')

plt.yscale("log")
plt.xlabel("1/c = N/d")
plt.ylabel("Generalization Error")
plt.legend(['r = 25','r = 50','r = 100','r = 150'])
plt.savefig(path_figures+"SVHN_error_nonIdentical_ranks.pdf", bbox_inches='tight', facecolor='white', dpi = 300, format = 'pdf')